<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/si_rec_cont.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import json

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from IPython.display import display

# Configurações do matplotlib
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('mathtext', fontset='cm')

# Sistema de equações diferenciais do modelo SIR com natalidade, mortalidade e doença
def sir_continuo(beta, gamma, nu, mu, alpha, Lambda, N0, I0, R0, T):
    S0 = max(0, N0 - I0 - R0)
    y0 = [S0, I0, R0]
    t_span = (0, T)
    t_eval = np.arange(0, T + 0.01, 0.01)  # subintervalos de 0.01

    sol = solve_ivp(
        fun=lambda t, y: sir_ode(t, y, beta, gamma, nu, mu, alpha, Lambda),
        t_span=t_span,
        y0=y0,
        method='RK45',
        t_eval=t_eval,
        rtol=1e-6,
        atol=1e-8
    )

    S, I, R = sol.y
    N = S + I + R
    return sol.t, S, I, R, N

# Função de plotagem com os valores de equilíbrio numérico
def plot_sir_continuo(beta, gamma, nu, mu, alpha, Lambda, N0, I0, R0, T):
    t, S, I, R, N = sir_continuo(beta, gamma, nu, mu, alpha, Lambda, N0, I0, R0, T)

    I_eq = I[-1]
    N_eq = N[-1]

    plt.figure(figsize=(14, 10), dpi=200)

    # Gráfico de I(t)
    plt.subplot(2, 2, 1)
    plt.plot(t, I, color='blue')
    plt.xlabel('$t$ (dias)')
    plt.ylabel('$I(t)$')
    plt.title('Série temporal de $I(t)$')
    plt.grid(True)
    plt.text(0.7 * T, 0.9 * max(I), f'$I^* = {I_eq:.2f}$', fontsize=12, bbox=dict(facecolor='white', alpha=0.7))

    # Gráfico de N(t)
    plt.subplot(2, 2, 2)
    plt.plot(t, N, color='blue')
    plt.xlabel('$t$ (dias)')
    plt.ylabel('$N(t)$')
    plt.title('Série temporal de $N(t)$')
    plt.grid(True)
    plt.text(0.7 * T, 0.9 * max(N), f'$N^* = {N_eq:.2f}$', fontsize=12, bbox=dict(facecolor='white', alpha=0.7))

    # Diagrama de fase: I vs N
    plt.subplot(2, 2, 3)
    plt.plot(N, I, color='blue')
    plt.plot(N_eq, I_eq, 'ro', label='Equilíbrio')
    plt.xlabel('$N(t)$')
    plt.ylabel('$I(t)$')
    plt.title('Diagrama de fase: $I(t)$ vs. $N(t)$')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Sliders dos parâmetros
beta_slider = widgets.FloatSlider(value=0.6, min=0.01, max=1.5, step=0.001, description='beta', readout_format='.4f')
gamma_slider = widgets.FloatSlider(value=0.113, min=0.001, max=1.0, step=0.001, description='gamma', readout_format='.4f')
nu_slider = widgets.FloatSlider(value=0.000, min=0.0, max=0.1, step=0.0001, description='nu', readout_format='.4f')
mu_slider = widgets.FloatSlider(value=0.001, min=0.0001, max=0.1, step=0.0001, description='mu', readout_format='.4f')
alpha_slider = widgets.FloatSlider(value=0.0223, min=0.001, max=0.1, step=0.0001, description='alpha', readout_format='.4f')
Lambda_slider = widgets.FloatSlider(value=13, min=0, max=100, step=1, description='Lambda')
N0_slider = widgets.IntSlider(value=10000, min=100, max=100000, step=10, description='N0')
I0_slider = widgets.IntSlider(value=10, min=0, max=500, step=1, description='I0')
R0_slider = widgets.IntSlider(value=0, min=0, max=500, step=1, description='R0')
T_slider = widgets.IntSlider(value=5*365, min=50, max=50*365, step=10, description='T')

# Interface interativa
ui = widgets.VBox([
    beta_slider, gamma_slider, nu_slider, mu_slider, alpha_slider,
    Lambda_slider, N0_slider, I0_slider, R0_slider, T_slider
])

out = widgets.interactive_output(plot_sir_continuo, {
    'beta': beta_slider,
    'gamma': gamma_slider,
    'nu': nu_slider,
    'mu': mu_slider,
    'alpha': alpha_slider,
    'Lambda': Lambda_slider,
    'N0': N0_slider,
    'I0': I0_slider,
    'R0': R0_slider,
    'T': T_slider
})

display(ui, out)


Output()

In [24]:
# Function to clean the notebook metadata
def clean_notebook_metadata(notebook_path):
    with open(notebook_path, 'r') as f:
        notebook_data = json.load(f)

    if 'metadata' in notebook_data and 'widgets' in notebook_data['metadata']:
        for widget in notebook_data['metadata']['widgets'].values():
            if 'state' in widget:
                del widget['state']

    with open(notebook_path, 'w') as f:
        json.dump(notebook_data, f, indent=2)

# Clean the metadata before saving the notebook
#notebook_path = os.path.realpath("si_rec_cont.ipynb")
notebook_path = os.path.realpath("/content/drive/MyDrive/Colab Notebooks/si_rec_cont.ipynb")
clean_notebook_metadata(notebook_path)